In [1]:
# example making new folder with os
import os
os.makedirs('Data/',exist_ok=True) # Confirm folder created
os.listdir("Data/")


['.ipynb_checkpoints',
 'final_tmdbdata2000.csv.gz',
 'final_tmdbdata2001.csv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_apiresults2000.json',
 'tmdb_apiresults2001.json']

In [63]:
import pandas as pd
import numpy as np
import seaborn as sns

In [3]:
import json
with open('/Users/gleal/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [4]:
import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key']

In [5]:
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"

In [6]:
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"

In [7]:
akas = pd.read_csv('title-akas-us-only.csv', low_memory=False)

In [8]:
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)

In [9]:
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

In [10]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [11]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1991
1,tt0000002,5.8,265
2,tt0000003,6.5,1860
3,tt0000004,5.5,178
4,tt0000005,6.2,2646


In [12]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,\N,imdbDisplay,\N,0
1,tt0000002,7,The Clown and His Dogs,US,\N,\N,literal English title,0
2,tt0000005,10,Blacksmith Scene,US,\N,imdbDisplay,\N,0
3,tt0000005,1,Blacksmithing Scene,US,\N,alternative,\N,0
4,tt0000005,6,Blacksmith Scene #1,US,\N,alternative,\N,0


In [13]:
akas = akas.replace({'\\N':np.nan})

In [14]:
basics = basics.replace({'\\N':np.nan})


In [15]:
ratings = ratings.replace({'\\N':np.nan})

In [16]:
basics = basics.dropna(subset=['runtimeMinutes', 'genres'])

In [17]:
basics = basics.query("titleType == 'movie'")

In [18]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,NaN,100,"Documentary,News,Sport"
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"


In [19]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 387260 entries, 8 to 10087332
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          387260 non-null  object
 1   titleType       387260 non-null  object
 2   primaryTitle    387260 non-null  object
 3   originalTitle   387260 non-null  object
 4   isAdult         387260 non-null  object
 5   startYear       380694 non-null  object
 6   endYear         0 non-null       object
 7   runtimeMinutes  387260 non-null  object
 8   genres          387260 non-null  object
dtypes: object(9)
memory usage: 29.5+ MB


In [20]:
basics['startYear'].value_counts()

2017    14425
2018    14409
2019    14150
2016    14001
2015    13513
        ...  
1904        1
1897        1
2027        1
1896        1
1894        1
Name: startYear, Length: 131, dtype: int64

In [21]:
basics['startYear'] = basics['startYear'].replace('\\N', 0)
basics['startYear'] = basics['startYear'].astype(float)
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 387260 entries, 8 to 10087332
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          387260 non-null  object 
 1   titleType       387260 non-null  object 
 2   primaryTitle    387260 non-null  object 
 3   originalTitle   387260 non-null  object 
 4   isAdult         387260 non-null  object 
 5   startYear       380694 non-null  float64
 6   endYear         0 non-null       object 
 7   runtimeMinutes  387260 non-null  object 
 8   genres          387260 non-null  object 
dtypes: float64(1), object(8)
memory usage: 29.5+ MB


In [22]:
basics = basics.loc[basics["startYear"].between(2000, 2021)]

In [23]:
# Exclude movies that are included in the documentary category.
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]


In [24]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34801,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61113,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67487,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016.0,NaN,90,Drama
67665,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86792,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"


In [25]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics['tconst'].isin(akas['titleId'])
keepers

34801        True
61113        True
67487        True
67665        True
86792        True
            ...  
10087064     True
10087103    False
10087148     True
10087232    False
10087322    False
Name: tconst, Length: 139003, dtype: bool

In [26]:
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34801,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61113,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67487,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016.0,NaN,90,Drama
67665,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86792,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...,...
10086520,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019.0,NaN,74,Drama
10086915,tt9915872,movie,The Last White Witch,Boku no kanojo wa mahoutsukai,0,2019.0,NaN,97,"Comedy,Drama,Fantasy"
10087055,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama
10087064,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"


In [27]:
rat_keep =ratings['tconst'].isin(akas['titleId'])
rat_keep

0           True
1           True
2          False
3          False
4           True
           ...  
1338672    False
1338673    False
1338674    False
1338675    False
1338676    False
Name: tconst, Length: 1338677, dtype: bool

In [28]:
ratings = ratings[rat_keep]
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1991
1,tt0000002,5.8,265
4,tt0000005,6.2,2646
5,tt0000006,5.0,183
6,tt0000007,5.4,830
...,...,...,...
1338638,tt9916200,8.1,233
1338639,tt9916204,8.2,267
1338646,tt9916348,8.3,18
1338647,tt9916362,6.4,5478


In [29]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81896 entries, 34801 to 10087148
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          81896 non-null  object 
 1   titleType       81896 non-null  object 
 2   primaryTitle    81896 non-null  object 
 3   originalTitle   81896 non-null  object 
 4   isAdult         81896 non-null  object 
 5   startYear       81896 non-null  float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  81896 non-null  object 
 8   genres          81896 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.2+ MB


In [30]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504953 entries, 0 to 1338652
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         504953 non-null  object 
 1   averageRating  504953 non-null  float64
 2   numVotes       504953 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.4+ MB


In [31]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1452564 entries, 0 to 1452563
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1452564 non-null  object
 1   ordering         1452564 non-null  int64 
 2   title            1452564 non-null  object
 3   region           1452564 non-null  object
 4   language         4018 non-null     object
 5   types            981678 non-null   object
 6   attributes       47016 non-null    object
 7   isOriginalTitle  1451222 non-null  object
dtypes: int64(1), object(7)
memory usage: 88.7+ MB


In [32]:
## Save current dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [33]:
## Save current dataframe to file.
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

In [34]:
## Save current dataframe to file.
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)

In [35]:
# Open saved file and preview again
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016.0,NaN,90,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"


In [36]:
# Open saved file and preview again
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1991
1,tt0000002,5.8,265
2,tt0000005,6.2,2646
3,tt0000006,5.0,183
4,tt0000007,5.4,830


In [37]:
# Open saved file and preview again
akas = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [38]:
import os, time,json
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'final_tmdbdata2000.csv.gz',
 'final_tmdbdata2001.csv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_apiresults2000.json',
 'tmdb_apiresults2001.json']

In [39]:
# def write_json(new_data, filename): 
#     """Appends a list of records (new_data) to a json file (filename). 
#     Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
#     with open(filename,'r+') as file:
#         # First we load existing data into a dict.
#         file_data = json.load(file)
#         ## Choose extend or append
#         if (type(new_data) == list) & (type(file_data) == list):
#             file_data.extend(new_data)
#         else:
#              file_data.append(new_data)
#         # Sets file's current position at offset.
#         file.seek(0)
#         # convert back to json.
#         json.dump(file_data, file)

In [40]:
# def get_movie_with_rating(movie_id):
#     """Adapted from source = https://github.com/celiao/tmdbsimple"""
#    #Get the movie object for the current id
#     movie = tmdb.Movies(movie_id)
    
#    #Save the .info .releases dictionaries
#     info = movie.info()
    
#     releases = movie.releases()
#    #Loop through countries in releases
#     for c in releases['countries']:
#         #if the country abbreviation==US
#         if c['iso_3166_1'  ] == 'US':
#             ##save a "certification" key in info with the certification
#             info['certification'] = c['certification']
#     return info 

In [41]:
def get_movie_with_rating(movie_id):
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    # save the .info .releases dictionaries
    movie_info = movie.info()

    releases = movie.releases()
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            ## save a "certification" key in the info dict with the certification
            movie_info['certification'] = c['certification']
    return movie_info

In [42]:
def write_json(new_data, filename): 
    """Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""

    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [43]:
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv('Data/title_basics.csv.gz')
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016.0,NaN,90,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"


In [44]:
YEARS_TO_GET = [2000,2001]

In [45]:
errors = [ ]

In [46]:
# for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
#     #Defining the JSON file to store results for year
#     JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
#     # Check if file exists
#     file_exists = os.path.isfile(JSON_FILE)

#     # If it does not exist: create it
#     if file_exists == False:
#     # save an empty dict with just "imdb_id" to the new json file.
#         with open(JSON_FILE,'w') as f:
#             json.dump([{'imdb_id':0}],f)

#     #Saving new year as the current df
#     df = basics.loc[ basics['startYear']==YEAR].copy()
#     # saving movie ids to list
#     movie_ids = df['tconst'].copy()

#     # Load existing data from json into a dataframe called "previous_df"
#     previous_df = pd.read_json(JSON_FILE)

#     # filter out any ids that are already in the JSON_FILE
#     movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

#     #Get index and movie id from list
#     # INNER Loop
#     for movie_id in tqdm_notebook(movie_ids_to_get,
#                                   desc=f'Movies from {YEAR}',
#                                   position=1,
#                                   leave=True):
#         try:
#             # Retrieve then data for the movie id
#             temp = get_movie_with_rating(movie_id)  
#             # Append/extend results to existing file using a pre-made function
#             write_json(temp,JSON_FILE)
#             # Short 20 ms sleep to prevent overwhelming server
#             time.sleep(0.02)
            
#         except Exception as e:
#             errors.append([movie_id, e])

#     final_year_df = pd.read_json(JSON_FILE)
#     final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

In [47]:
for YEAR in tqdm_notebook(YEARS_TO_GET,desc='YEARS',
                          position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_apiresults{YEAR}.json'

    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)


    # If it does not exist: create it
    if file_exists == False:
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)



    #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()#.to_list()


    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)


    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]


    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        # Attempt to retrieve then data for the movie id
        try:
            temp = get_movie_with_rating(movie_id)  #This uses your pre-made function
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)

        # If it fails,  make a dict with just the id and None for certification.
        except Exception as e:
            errors.append([movie_id, e])


    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdbdata{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/209 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/233 [00:00<?, ?it/s]

In [48]:
print(f"- Total errors: {len(errors)}")

- Total errors: 442


In [49]:
#Avengers Test
test = get_movie_with_rating("tt0848228") #put your function name here
test

{'adult': False,
 'backdrop_path': '/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 121.52,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path'

In [50]:
#The Notebook Test
test = get_movie_with_rating("tt0332280") #put your function name here
test

{'adult': False,
 'backdrop_path': '/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg',
 'belongs_to_collection': None,
 'budget': 29000000,
 'genres': [{'id': 10749, 'name': 'Romance'}, {'id': 18, 'name': 'Drama'}],
 'homepage': 'http://www.newline.com/properties/notebookthe.html',
 'id': 11036,
 'imdb_id': 'tt0332280',
 'original_language': 'en',
 'original_title': 'The Notebook',
 'overview': "An epic love story centered around an older man who reads aloud to a woman with Alzheimer's. From a faded notebook, the old man's words bring to life the story about a couple who is separated by World War II, and is then passionately reunited, seven years later, after they have taken different paths.",
 'popularity': 44.1,
 'poster_path': '/rNzQyW4f8B8cQeg7Dgj3n6eT5k9.jpg',
 'production_companies': [{'id': 12,
   'logo_path': '/5ThIuO93vsk47oexKTSdfKEr7EC.png',
   'name': 'New Line Cinema',
   'origin_country': 'US'},
  {'id': 1565, 'logo_path': None, 'name': 'Avery Pix', 'origin_country': 'US'},
  {'id': 2605